## Data Audit

Purpose:
- Understand structure and quality of the raw dataset
- Identify issues to fix during cleaning

In [1]:
import sys
sys.executable


'/home/ruan/dev/supply-chain-data-cleaning/.venv/bin/python'

In [2]:
import pandas as pd
pd.__version__


'3.0.0'